In [1]:
import numpy as np
import pandas as pd
import collections
from string import ascii_lowercase

import re

In [2]:
f = open('inputs/day07_test.txt')
data = f.read()

In [35]:
def flatten(l):
    return list(flatten_generator(l))
            
def flatten_generator(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten_generator(el)
        else:
            yield el

class InstructionTree:
    
    def __init__(self, instructions, workers=1, base_seconds=60):
        self.data = data
        self.time = 0
        self.workers = workers
        self.node_names = list(pd.Series(flatten(re.findall('(?<= )[A-Z](?= )', data))).sort_values().unique())
        self.nodes = {}
        for n in self.node_names:
            self.nodes[n] = Node(n, base_seconds)
        self.instructions = data.split('\n')
        self.completed = False
        self.completed_nodes = []
        self.next_available_nodes = []
        self.active_nodes = []
    
    def parse_line(self, line):
        names = re.findall('(?<= )[A-Z](?= )', line)    
        return names[0], names[1]
    
    def build_tree(self):
        for line in self.instructions:
            n0, n1 = self.parse_line(line)
            self.nodes[n0].add_child(self.nodes[n1])
            self.nodes[n1].add_parent(self.nodes[n0])
        for nm in self.node_names:
            node = self.nodes[nm]
            if not node.has_parent():
                self.next_available_nodes.append(node)
        self.next_available_nodes.sort()
        self.current = self.next_available_nodes.pop(0)
        self.active_nodes.append(self.current)
        
    def completed_parents(self, node):
        return all([parent in self.completed_nodes for parent in node.parents])
            
    def find_next_available_nodes(self, node):
        next_list = []
        # Determine next available nodes
        # Check that the current node has children to add
        if node.has_children():
            # only add children if all of their parents have been completed
            for child in node.children:
                if self.completed_parents(child):
                    next_list.append(child)
        # resort the list
        return next_list
        
    def part1_increment(self):
        # Complete the current node
        self.completed_nodes.append(self.current)

        # Find next available nodes
        self.next_available_nodes.append(self.find_next_available_nodes(self.current))
        self.next_available_nodes = flatten(self.next_available_nodes)
        self.next_available_nodes.sort()    
        print(self.current, self.next_available_nodes)
        
        # Change the current node to the next node
        if len(self.next_available_nodes) > 0:
            self.current = self.next_available_nodes.pop(0)
        else:
            self.completed = True
            
    def part2_increment(self):
        self.time += 1
        for current in self.active_nodes:
            current.increment(1)
            if current.solved:
                self.completed_nodes.append(current)
                self.active_nodes.remove(current)
        
        for an in self.active_nodes:
            self.next_available_nodes.append(self.find_next_available_nodes(an))
        
        self.next_available_nodes = self.format_available_nodes(self.next_available_nodes)
            
        while len(self.active_nodes) < self.workers:
            x = self.next_available_nodes.pop(0)
            self.active_nodes.append(x)
            
    def format_available_nodes(self, node_list):
        node_list = list(set(flatten(node_list)))
        node_list.sort()
        return node_list
        
class Node:

    def __init__(self, name, base_seconds=60):
        self.name = name
        self.parents = []
        self.children = []
        self.time = 0
        self.time_to_solve = base_seconds+ascii_lowercase.upper().index(name)+1
        self.solved = False
    
    def __repr__(self):
        return self.name
    
    def __gt__(self, other):
        self.name > other.name
        
    def __eq__(self, other):
        return self.name == other.name

    def __lt__(self, other):
        return self.name < other.name
    
    def add_parent(self, parent):
        self.parents.append(parent)
        
    def has_parent(self):
        return len(self.parents) > 0
        
    def add_child(self, child):
        self.children.append(child)
        
    def has_children(self):
        return len(self.children) > 0
    
    def increment(self, seconds):
        self.time += 1
        if self.time >= self.time_to_solve:
            self.solved = True
        

# Part 1

In [36]:
tree = InstructionTree(data)
tree.build_tree()

In [37]:
while not tree.completed:
    tree.part1_increment()
''.join([n.name for n in tree.completed_nodes])

C [A, F]
A [B, D, F]
B [D, F]
D [F]
F [E]
E []


'CABDFE'

# Part 2

In [44]:
tree = InstructionTree(data, base_seconds=0)
tree.build_tree()

In [57]:
tree.completed_nodes[0].time_to_solve

3

In [40]:
tree.part2_increment()

In [41]:
tree.active_nodes

[C]

In [47]:
tree.part2_increment()
print(tree.active_nodes, tree.completed_nodes, tree.next_available_nodes)

IndexError: pop from empty list

In [34]:
while not tree.completed:
    tree.part2_increment()
''.join([n.name for n in tree.completed_nodes])

AttributeError: 'list' object has no attribute 'increment'

In [11]:
tree.completed_nodes

[]